In [1]:
from BeautifulSoup import BeautifulSoup
import urlparse
import urllib2
import pandas as pd
import numpy as np
import cPickle
import matplotlib.pyplot as plt

In [79]:
class HTMLdata(object):
    def __init__(self, base = 'http://espn.go.com/mens-college-basketball/statistics/team/_/stat/'):
        self.base = base
        self.glossarydict = None
        
    yearList = [2017]
    metricList = ['rebounds', 'free-throws', '3-points', 'assists', 'steals', 'blocks']
    countList = map(str, range(41, 361, 40))

    def urlsCurrent(self):
        count0 = map(lambda metric: self.base + '{0}/seasontype/2/'.format(metric), self.metricList)
        urls = [map(lambda count: url + 'count/{0}'.format(count), self.countList) for url in count0]
        urls = sum(urls, []) 
        return count0 + urls # should result in 54 urls 6metrics * 9count pages
    
    def urlsPast(self):
        count0 = map(lambda metric: self.base + '{0}/seasontype/2/'.format(metric), self.metricList)
        yearUrl = [map(lambda year: url + 'year/{0}'.format(year), self.yearList) for url in count0]
        result1 = sum(yearUrl, [])
        countUrl = [map(lambda count: url + '/count/{0}'.format(count), self.countList) for url in result1]
        result2 = sum(countUrl, [])
        return result1 + result2 ### len should be 648 = (6(metric groups * 9(count pages) * 12(year groups))
    
    def extract_htmldata(self, urlList):
        dataList = [[] for i in range(0, len(urlList) )]
        for i in range(0,len(urlList)):
            html = urllib2.urlopen(urlList[i]).read()
            [dataList[i].append(tr.findAll(text=True)) for tr in BeautifulSoup(html).findAll('tr')] ## get each row of data
            dataList[i].append([BeautifulSoup(html).title.string[ :4]]) # get the season and add to the end of list
        return dataList

In [80]:
parser = HTMLdata()
test = parser.urlsCurrent()
urlList = parser.urlsPast()
dataList = parser.extract_htmldata(urlList)

In [81]:
cPickle.dump(dataList, open('dataList16-17.p', 'wb'))

In [82]:
class DATA(object):
    def __init__(self):
        pass
    
    def extractTable(self,dataList):
        for data in dataList:
            if len(data[0]) < len(data[-2]):    # subheader distinguishes between Per game and total; after subheader was removed some column names are identical and 
                data[1][4], data[1][5] = data[1][4] + 'PG', data[1][5] + 'PG'   # need to be changed to reflect Per game
                data[1][6], data[1][7] = data[1][6] + 'Total', data[1][7] + 'Total' # and total
            [data.remove(row) for row in data[:-1] if len(row) < len(data[-2])] # data[:-1] to exclude ['2013']
            header = data.pop(0)
            [data.remove(row) for row in data if row[0] == 'RK']
            data.insert(0,header)
        return dataList

    def addSeason(self, dataList):
        for data in dataList:   # dataList contains a list with data from each url
            data[0].append('Season')    # add column name to the end of header
            [data[i].append(data[-1][0]) for i in range(1,len(data)-1)] # data[-1][i] = ['2013']' add the season value to th end fo each row
            data.pop(-1)
        return dataList

    def combined(self, dataList):
        dataList.sort()     # dataList contains a list with data from each url
        header = dict((tuple(x[0]), x[1:]) for x in dataList).keys() # get unique headers(first element of each list) by passing them as keys ato a dictionary
        combined = [[] for i in range(0, len(header)) ]
        for i in range(0, len(header)):
            [combined[i].append(data) for data in dataList if data[0] == list(header[i])]   # based on its header group the list into a new list   
        return combined ## each combined list has 117lists = 13season*9pages

    def dfList(self, combined):
        dfList = [[] for i in range(0, len(combined))]
        for i in range(0, len(combined)): # each list in combined has lists with identical headers
            for data in combined[i]:
        # have to map the data as it should be a list type but is ResultSet from Beautifulsoup
                dfList[i].append(pd.DataFrame(map(list, data)[1:], columns = map(list, data)[0]) )
        for i in range(0, len(dfList)):
            dfList[i] = pd.concat(dfList[i]) #, ignore_index = True) - shoudld be added to ignore headers but crashes
        return dfList

    def mergeDF(self, df):
        for i in range(1,len(df)):
            #colsToUse = (df[i].columns - df[0].columns).tolist() ## to avoid duplicating columns when merging; some columns repeat in the dataframes
            colsToUse = list(set(dfList[i].columns) - set(dfList[0].columns))
            colsToUse.append('TEAM'); colsToUse.append('Season') # need to add TEAM and Season to facilitate merging on those columns
            
            mergeDF = pd.merge(df[0], df[i][colsToUse], how='left', on=['TEAM', 'Season'] )
            df[0] = mergeDF
        mergeDF = mergeDF.drop(['RK'], 1)
        return mergeDF
    

In [83]:
dataList = cPickle.load(open('dataList16-17.p', 'rb'))
data = DATA()
table = data.extractTable(dataList) 
addSeason = data.addSeason(table)
combined = data.combined(addSeason)
dfList = data.dfList(combined)
mDF = data.mergeDF(dfList)

In [84]:
mDF

,TEAM,GP,AST,APG,TO,TOPG,AST/TO,Season,BLK/PF,PF,...,3PMPG,3PAPG,FG%,2PM,3P%,FTAPG,FTATotal,FT%,FTMPG,FTMTotal
0,UCLA,33,709,21.5,381,11.5,1.86,2016,0.33,542,...,9.9,24,.595,786,.405,17.5,576,.745,13.0,429
1,North Carolina,34,619,18.2,405,11.9,1.53,2016,0.18,605,...,7.2,20,.526,809,.366,22.1,753,.705,15.6,531
2,Purdue,32,581,18.2,419,13.1,1.39,2016,0.16,504,...,9.0,22,.557,606,.406,20.0,639,.764,15.3,488
3,Lipscomb,33,599,18.2,500,15.2,1.20,2016,0.15,702,...,10.3,28,.561,599,.366,24.6,811,.723,17.8,586
4,Canisius,33,579,17.5,436,13.2,1.33,2016,0.10,643,...,9.2,25,.530,606,.374,19.3,636,.734,14.2,467
5,William &amp; Mary,31,543,17.5,388,12.5,1.40,2016,0.27,566,...,9.1,24,.569,604,.382,22.2,689,.692,15.4,477
6,Fort Wayne,31,543,17.5,422,13.6,1.29,2016,0.22,559,...,10.7,26,.580,665,.407,17.1,531,.734,12.6,390
7,Iowa,32,555,17.3,439,13.7,1.26,2016,0.21,569,...,8.6,23,.517,621,.371,22.1,707,.700,15.5,495
8,Savannah State,29,501,17.3,431,14.9,1.16,2016,0.16,571,...,13.5,41,.507,441,.332,22.7,659,.648,14.7,427
9,Creighton,34,587,17.3,424,12.5,1.38,2016,0.18,563,...,8.7,22,.580,755,.399,16.9,573,.682,11.5,391


In [86]:
mDF.to_csv('dataList16-17.csv', sep='\t', encoding='utf-8')